In [1]:
# !pip install transformers==3.1.0

In [2]:
import pandas as pd

dataset_number = 1

# Lacafe
# text = 'txt'
# label = 'has_anger'
# train_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/Lacafe/df_dataset_test_{dataset_number}.csv')

# Fortuna (Updated)
# text = 'text'
# label = 'hatespeech_comb'
# train_data = pd.read_csv(f'./Datasets/FortunaUpdated/2019-05-28_portuguese_hate_speech_binary_classification_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/FortunaUpdated/2019-05-28_portuguese_hate_speech_binary_classification_test_{dataset_number}.csv')

# OffComBr
# text = 'text'
# label = 'offensive'
# train_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/OffComBR/OffComBR2_test_{dataset_number}.csv')

# HateBR
# text = 'instagram_comments'
# label = 'offensive_language'
# train_data = pd.read_csv(f'./Datasets/HateBR/HateBR_train_{dataset_number}.csv')
# test_data = pd.read_csv(f'./Datasets/HateBR/HateBR_test_{dataset_number}.csv')

In [3]:
from datasets import Dataset, DatasetDict

train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

raw_datasets = DatasetDict({'train': train_data, 'test': test_data})

train_data
test_data
raw_datasets

2022-07-21 03:45:00.078601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-21 03:45:00.078621: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


DatasetDict({
    train: Dataset({
        features: ['txt', 'has_anger'],
        num_rows: 5369
    })
    test: Dataset({
        features: ['txt', 'has_anger'],
        num_rows: 2302
    })
})

In [4]:
# Tratamento básico dos dados

train_texts = raw_datasets['train'][text]
train_labels = raw_datasets['train'][label]
test_texts = raw_datasets['test'][text]
test_labels = raw_datasets['test'][label]

print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("TestTexts Length: ", len(test_texts))
print("TestLabels Length: ", len(test_labels))

TrainTexts Length:  5369
TrainLabels Length:  5369
TestTexts Length:  2302
TestLabels Length:  2302


In [5]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1, stratify=train_labels)

In [6]:
print("TrainTexts Length: ", len(train_texts))
print("TrainLabels Length: ", len(train_labels))
print("ValidationTexts Length: ", len(val_texts))
print("ValidationLabels Length: ", len(val_labels))

TrainTexts Length:  4832
TrainLabels Length:  4832
ValidationTexts Length:  537
ValidationLabels Length:  537


In [7]:
print('Quantidade classes positivas (treino): ', sum(train_labels))
print('Quantidade classes negativas (treino): ', len(train_labels) - sum(train_labels))
print('Quantidade classes positivas (validação): ', sum(val_labels))
print('Quantidade classes negativas (validação): ', len(val_labels) - sum(val_labels))

Quantidade classes positivas (treino):  2416
Quantidade classes negativas (treino):  2416
Quantidade classes positivas (validação):  268
Quantidade classes negativas (validação):  269


In [8]:
from transformers import pipeline, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', max_len=512)
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', max_len=512)
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', max_len=512)

# classifier = pipeline("zero-shot-classification", model='neuralmind/bert-base-portuguese-cased', tokenizer=tokenizer)
# classifier = pipeline("zero-shot-classification", model='bert-base-multilingual-cased', tokenizer=tokenizer)
classifier = pipeline("zero-shot-classification", model='dccuchile/bert-base-spanish-wwm-cased', tokenizer=tokenizer)

candidate_labels = ['não tóxico', 'tóxico']

results = classifier(test_texts, candidate_labels)

cm = {'true_positive': 0, 'true_negative': 0, 'false_positive': 0, 'false_negative': 0}
for i in range(len(results)):
    result = results[i]
    
    if result['labels'][0] == 'tóxico':
        predicted_label = 1
    else:
        predicted_label = 0

    if (test_labels[i] == predicted_label):
        if test_labels[i] == 1:
            cm['true_positive'] += 1
        else:
            cm['true_negative'] += 1
    else:
        if test_labels[i] == 0:
            cm['false_positive'] += 1
        else:
            cm['false_negative'] += 1
        
print(cm)
precision = cm['true_positive'] / (cm['true_positive'] + cm['false_positive'])
recall = cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])
fscore = 2 * ((precision * recall) / (precision + recall))
acertos = cm['true_positive'] + cm['true_negative']
total = len(test_data)
print(f'Acertos: {acertos}')
print(f'Total: {total}')
print(f'Accuracy: {str((acertos / total) * 100).replace(".", ",")}')
print(f'Precision: {str((precision * 100)).replace(".", ",")}')
print(f'Recall: {str((recall * 100)).replace(".", ",")}')
print(f'Fscore: {str((fscore * 100)).replace(".", ",")}')

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

KeyboardInterrupt: 